<a href="https://colab.research.google.com/github/damilojohn/Text-Descrambling/blob/main/GPT2/Text_Descrambling_gpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 !pip install -qU transformers datasets nvidia-ml-py3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 21.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 25.7 MB/s eta 0:00:00


In [ ]:
import os
import time
import datetime
from google.colab import drive

import pandas as pd
import seaborn as sns
import numpy as np
import random

import matplotlib.pyplot as plt
%matplotlib inline

import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler
torch.manual_seed(42)

from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config, GPT2LMHeadModel
from transformers import AdamW, get_linear_schedule_with_warmup


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
train_path = '/content/drive/MyDrive/Colab datasets/Text-Deshuffling/train (8).csv'
test_path = '/content/drive/MyDrive/Colab datasets/Text-Deshuffling/test (9).csv'
val_path= '/content/drive/MyDrive/Colab datasets/Text-Deshuffling/val (1).csv'
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
val_df = pd.read_csv(val_path)


Mounted at /content/drive


In [ ]:
train_df.head()

,text,label,id
0,"presented here Furthermore, naive improved. im...","Furthermore, the naive implementation presente...",0
1,vector a in a form vector multidimensional spa...,Those coefficients form a vector in a multidim...,1
2,compatible of The model with recent is model s...,The model is compatible with a recent model of...,2
3,but relevance outlined. hemodynamics its based...,"The model is based on electrophysiology, but i...",3
4,of transitions lever-like involve reorientatio...,Conformational transitions in macromolecular c...,4


In [ ]:
train_df['length'] = train_df['text'].apply(lambda x : len(x.split(' ')))

In [ ]:
train_df

,text,label,id,length
0,"presented here Furthermore, naive improved. im...","Furthermore, the naive implementation presente...",0,9
1,vector a in a form vector multidimensional spa...,Those coefficients form a vector in a multidim...,1,10
2,compatible of The model with recent is model s...,The model is compatible with a recent model of...,2,12
3,but relevance outlined. hemodynamics its based...,"The model is based on electrophysiology, but i...",3,14
4,of transitions lever-like involve reorientatio...,Conformational transitions in macromolecular c...,4,12
...,...,...,...,...
39996,for pose a clutter. estimation autonomous mani...,Object pose estimation is a crucial prerequisi...,39996,15
39997,objects added warehouses bin-picking present s...,Real-world bin-picking settings such as wareho...,39997,15
39998,validation The proposed real-world on is metho...,The proposed method is evaluated on a syntheti...,39998,14
39999,a between modes. statistics survival qualitati...,This breakdown is associated with a crossover ...,39999,15


In [ ]:
max(train_df['length'])

15

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>') #gpt2-medium'

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
print("The max model length is {} for this model, although the actual embedding size for GPT small is 768".format(tokenizer.model_max_length))
print("The beginning of sequence token {} token has the id {}".format(tokenizer.convert_ids_to_tokens(tokenizer.bos_token_id), tokenizer.bos_token_id))
print("The end of sequence token {} has the id {}".format(tokenizer.convert_ids_to_tokens(tokenizer.eos_token_id), tokenizer.eos_token_id))
print("The padding token {} has the id {}".format(tokenizer.convert_ids_to_tokens(tokenizer.pad_token_id), tokenizer.pad_token_id))

The max model length is 1024 for this model, although the actual embedding size for GPT small is 768
The beginning of sequence token <|startoftext|> token has the id 50257
The end of sequence token <|endoftext|> has the id 50256
The padding token <|pad|> has the id 50258


In [ ]:
batch_size = 2 #gpt2 is a very large model 
#creating a standard pytorch dataset 
class GPT2dataset(Dataset):
  def __init__(self,df,tokenizer,gpt_type='gpt2',max_length=20):
    self.tokenizer = tokenizer
    self.input_ids = []
    self.attn_masks = []
    self.label_ids = []

    for input,label in zip(df['text'],df['label']):
       inputs = tokenizer('<|startoftext|>'+ input + '<|endoftext|>', truncation=True, max_length=max_length, padding="max_length")
       labels = tokenizer('<|startoftext|>' + label + '<|endoftext|>',truncation=True,max_length=max_length,padding='max_length')
       self.input_ids.append(torch.tensor(inputs['input_ids']))
       self.attn_masks.append(torch.tensor(inputs['attention_mask']))
       self.label_ids.append(torch.tensor(labels['input_ids']))
  def __len__(self):
    return len(self.input_ids)
  def __getitem__(self,idx):
    return self.input_ids[idx],self.attn_masks[idx],self.label_ids[idx]


In [ ]:
train_dataset = GPT2dataset(train_df,tokenizer,max_length=20)
val_dataset = GPT2dataset(val_df,tokenizer,max_length=20)


In [ ]:
#creating dataloaders 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [ ]:
configuration = GPT2Config.from_pretrained('gpt2',output_hidden_states=False)
model = GPT2LMHeadModel.from_pretrained('gpt2',config=configuration)
model.resize_token_embeddings(len(tokenizer))
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
#optimizer
# some parameters I cooked up that work reasonably well

epochs = 5
learning_rate = 5e-4
warmup_steps = 1e2
epsilon = 1e-8

# this produces sample output every 100 steps
sample_every = 100
optimizer = AdamW(model.parameters(),
                  lr = learning_rate,
                  eps = epsilon
                )

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
# This changes the learning rate as the training loop progresses
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = warmup_steps, 
                                            num_training_steps = total_steps)

In [ ]:
next(iter(train_dataloader))

[tensor([[50257, 20358,   257,  4981,   734,  4693,    13,   319,  1022,  1848,
          30135,   338, 15974,   262,  2312,  3965,  3292,    12,  2364,   329],
         [50257, 32861,    13,   287,   284,   554,   356,   670,    11,  4886,
          10618,   290,  1660,  4031,   428, 50256, 50258, 50258, 50258, 50258]]),
 tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]]),
 tensor([[50257,  4711,  4981,  1848,   329,   257,  3292,    12,  2364,  1022,
            262, 30135,   338,  1368,  3965,   319,   734, 15974,  4693,    13],
         [50257,   818,   428,   670,    11,   356,  4031,   284, 10618,   290,
           4886,  1660,   287,  5861,    13, 50256, 50258, 50258, 50258, 50258]])]

In [ ]:
#training loop 
total_train_loss = 0 
model.train()
device = torch.device("cuda")
model.cuda()
for epoch in range(epochs):
  #forward pass 
  for step,batch in enumerate(train_dataloader):
    b_input_ids = batch[0].to(device)
    b_attention_masks = batch[1].to(device)
    b_labels = batch[2].to(device)
    model.zero_grad()
    outputs = model(  b_input_ids,
                          labels=b_labels, 
                          attention_mask = b_attention_masks,
                          token_type_ids=None
                        )
    loss = outputs[0]
    batch_loss = loss.item()
    total_train_loss += batch_loss
    if step % sample_every == 0 and not step == 0:
            print('  Batch {:>5,}  of  {:>5,}. Loss: {:>5,}.'.format(step, len(train_dataloader), batch_loss,))

            model.eval()

            sample_outputs = model.generate(
                                    do_sample=True,   
                                    top_k=50, 
                                    max_length = 200,
                                    top_p=0.95, 
                                    num_return_sequences=1
                                )
            for i, sample_output in enumerate(sample_outputs):
                  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))
            
            model.train()

    loss.backward()

    optimizer.step()

    scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)       

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))


  Average training loss: 0.00

  Average training loss: 0.01

  Average training loss: 0.01

  Average training loss: 0.02

  Average training loss: 0.02

  Average training loss: 0.02

  Average training loss: 0.03

  Average training loss: 0.03

  Average training loss: 0.03

  Average training loss: 0.03

  Average training loss: 0.03

  Average training loss: 0.03

  Average training loss: 0.03

  Average training loss: 0.04

  Average training loss: 0.04

  Average training loss: 0.04

  Average training loss: 0.04

  Average training loss: 0.04

  Average training loss: 0.04

  Average training loss: 0.04

  Average training loss: 0.04

  Average training loss: 0.04

  Average training loss: 0.04

  Average training loss: 0.04

  Average training loss: 0.04

  Average training loss: 0.04

  Average training loss: 0.04

  Average training loss: 0.04

  Average training loss: 0.04

  Average training loss: 0.04

  Average training loss: 0.04

  Average training loss: 0.04

  Avera

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.



  Average training loss: 0.07

  Average training loss: 0.07
  Batch   100  of  20,001. Loss: 6.484448432922363.
0: This is-

  Average training loss: 0.07

  Average training loss: 0.07

  Average training loss: 0.07

  Average training loss: 0.07

  Average training loss: 0.07

  Average training loss: 0.07

  Average training loss: 0.07

  Average training loss: 0.07

  Average training loss: 0.07

  Average training loss: 0.07

  Average training loss: 0.07

  Average training loss: 0.07

  Average training loss: 0.07

  Average training loss: 0.07

  Average training loss: 0.07

  Average training loss: 0.07

  Average training loss: 0.07

  Average training loss: 0.07

  Average training loss: 0.07

  Average training loss: 0.07

  Average training loss: 0.07

  Average training loss: 0.07

  Average training loss: 0.07

  Average training loss: 0.07

  Average training loss: 0.07

  Average training loss: 0.07

  Average training loss: 0.07

  Average training loss: 0.08

  Ave

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.



  Average training loss: 0.10
  Batch   200  of  20,001. Loss: 6.919572353363037.
0: S. The the The The the of

  Average training loss: 0.10

  Average training loss: 0.10

  Average training loss: 0.10

  Average training loss: 0.10

  Average training loss: 0.10

  Average training loss: 0.10

  Average training loss: 0.10

  Average training loss: 0.10

  Average training loss: 0.10

  Average training loss: 0.10

  Average training loss: 0.10

  Average training loss: 0.10

  Average training loss: 0.10

  Average training loss: 0.10

  Average training loss: 0.10

  Average training loss: 0.10

  Average training loss: 0.10

  Average training loss: 0.10

  Average training loss: 0.10

  Average training loss: 0.10

  Average training loss: 0.10

  Average training loss: 0.10

  Average training loss: 0.10

  Average training loss: 0.10

  Average training loss: 0.10

  Average training loss: 0.10

  Average training loss: 0.10

  Average training loss: 0.10

  Average training 

...


  Average training loss: 9.94
  Batch 2,800  of  20,001. Loss: 2.8492820262908936.
0: However, digital the digital of to digital digital digital digital to digital digital digital digital digital

  Average training loss: 9.94

  Average training loss: 9.94

  Average training loss: 9.94

  Average training loss: 9.94

  Average training loss: 9.94

  Average training loss: 9.94

  Average training loss: 9.94

  Average training loss: 9.94

  Average training loss: 9.94

  Average training loss: 9.94

  Average training loss: 9.94

  Average training loss: 9.94

  Average training loss: 9.94

  Average training loss: 9.94

  Average training loss: 9.94

  Average training loss: 9.94

  Average training loss: 9.94

  Average training loss: 9.94

  Average training loss: 9.94

  Average training loss: 9.94

  Average training loss: 9.94

  Average training loss: 9.94

  Average training loss: 9.95

  Average training loss: 9.95

  Average training loss: 9.95

  Average training loss: 9.

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.



  Average training loss: 9.96
  Batch 2,900  of  20,001. Loss: 3.638192892074585.
0: This scheme employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employs employ

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.



  Average training loss: 9.98
  Batch 3,000  of  20,001. Loss: 3.747349500656128.
0: The resulting model of features model resulting model consists model resulting model model model model.

  Average training loss: 9.98

  Average training loss: 9.98

  Average training loss: 9.98

  Average training loss: 9.98

  Average training loss: 9.98

  Average training loss: 9.98

  Average training loss: 9.98

  Average training loss: 9.98

  Average training loss: 9.98

  Average training loss: 9.98

  Average training loss: 9.98

  Average training loss: 9.98

  Average training loss: 9.98

  Average training loss: 9.98

  Average training loss: 9.98

  Average training loss: 9.98

  Average training loss: 9.98

  Average training loss: 9.98

  Average training loss: 9.98

  Average training loss: 9.98

  Average training loss: 9.98

  Average training loss: 9.98

  Average training loss: 9.98

  Average training loss: 9.98

  Average training loss: 9.98

  Average training loss: 9.98

  A

In [ ]:
#monitoring GPU memory consumption 
from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")
print_gpu_utilization())

SyntaxError: ignored